In [ ]:
!pip install iapws

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iapws: filename=iapws-1.5.4-py3-none-any.whl size=117029 sha256=b620a47bfc12ab5c2b631cccf2ad2ceace53783bbd0d84c6e9360da19a3d4685
  Stored in directory: /root/.cache/pip/wheels/f8/8c/cc/e78b3e8b85e247119d5b9f9e7ab15f832fdb13a71caf83a8c7
Successfully built iapws


In [ ]:
import numpy as np
import iapws
from iapws import IAPWS97
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from scipy import linalg

In [ ]:
P_out = 0.20265 #MPa
P_in = P_out*2 #MPa
T_out = 273.15

r_interface = 0.03-0.0002
delr = np.linspace(0, r_interface, 15) #m
delw = np.linspace(r_interface, 0.03, 4) #m

total_nodes = np.unique(np.concatenate((delr, delw)))
dr = np.diff(total_nodes)
r_faces = np.hstack([0, total_nodes[:-1]+0.5*dr])
node_len = len(total_nodes)
print(total_nodes.size)

orig_T = [35+273.15]*node_len
T = np.array(orig_T.copy())
target_T = 10+273.15 #K
h_out = 956.25614 #W/m^2K
r_wall = 0.03-0.000175 #m
r_outer = 0.03 #m
delt = 10
delT = 0.5 #This is to define a delta temp specifically for the k' term.
del_space = total_nodes[1:] - total_nodes[:-1]

#Properties for Aluminum as a function of temp
def aluminum_properties(T):
   rho = (-2e-4*T**2) + (0.0282*T) +  2697.6
   if T >= 298:
      c_p = (0.3162*T) + 788.95
   else:
      c_p = (0.3162*298) + 788.95
   k = (2e-7*T**3) - (4e-4*T**2) + (0.2456*T) + 196.91
   return rho, c_p, k

def water_props(T):
    T = np.atleast_1d(T)
    rho_vals, cp_vals, k_vals = [], [], []
    for Ti in T:
        w = IAPWS97(P=P_in, T=Ti)
        rho_vals.append(w.rho)
        cp_vals.append(w.cp * 1000)  # kJ/kg·K → J/kg·K
        k_vals.append(w.k)
    rho_vals, cp_vals, k_vals = map(np.array, (rho_vals, cp_vals, k_vals))
    # Return scalars if input was scalar
    if rho_vals.size == 1:
        return float(rho_vals), float(cp_vals), float(k_vals)
    return rho_vals, cp_vals, k_vals

18


In [ ]:
def build_A_matrix(r, k):
    def harm(a, b):
        return 2*a*b/(a+b)

    A = np.zeros((node_len, node_len))
    b = np.zeros((node_len))

    for i in range(node_len):
        k_0 = 0
        k_2 = 0
        if i > 0:
          k_0 = harm(k[i], k[i-1])
        if i < node_len-1:
          k_2 = harm(k[i], k[i+1])

        if i < node_len-1:
            ri = total_nodes[i]
            rl = r_faces[i]
            rr = r_faces[i+1]

        if i == 0:
            A[i, i] = 1
            A[i, i+1] = -1
        elif i == node_len-1:
            A[i, i] = (k[i]/(dr[i-1]))+h_out
            A[i, i-1] = -k[i]/dr[i-1]
            b[-1] = h_out*T_out
        else:

            A[i, i-1] = ((2*k[i])/((dr[i]+dr[i-1])*dr[i-1])) - ((k[i+1]-k[i-1])/(dr[i]+dr[i-1])**2) - (k[i]/(total_nodes[i]*(dr[i]+dr[i-1])))
            A[i, i] = -((k[i]*2)/(dr[i]+dr[i-1]))*((1/dr[i])+(1/dr[i-1]))
            A[i, i+1] = ((2*k[i])/((dr[i]+dr[i-1])*dr[i])) + ((k[i+1]-k[i-1])/(dr[i]+dr[i-1])**2) + (k[i]/(total_nodes[i]*(dr[i]+dr[i-1])))

    #print(A, b)
    return A, b

In [81]:
def build_A_matrix_cd(r, k):
    """
    Node-centered central-difference operator for
    L[T] = k * ( d2T/dr2 + (1/r) dT/dr ) on non-uniform r.
    A is built so that A @ T approximates the diffusion operator.
    """
    N = len(r)
    A = np.zeros((N, N))
    b = np.zeros(N)

    # interior nodes
    for i in range(1, N - 1):
        rp = r[i + 1]
        rm = r[i - 1]
        ri = r[i]

        drp = rp - r[i]          # r_{i+1} - r_i
        drm = r[i] - rm          # r_i - r_{i-1}
        denom = rp - rm         # r_{i+1} - r_{i-1}

        # second-derivative central-difference on non-uniform grid
        c_ip = 2.0 / (denom * drp)
        c_im = 2.0 / (denom * drm)
        c_ii = - (c_ip + c_im)

        # first-derivative approx (central)
        d1_ip = 1.0 / denom * (1.0 / ri)
        d1_im = -1.0 / denom * (1.0 / ri)

        coeff_ip = c_ip + d1_ip
        coeff_im = c_im + d1_im
        coeff_ii = c_ii

        # Multiply by node k_i (your request: node-centered k, no harmonic mean)
        A[i, i + 1] = k[i] * coeff_ip
        A[i, i - 1] = k[i] * coeff_im
        A[i, i]     = k[i] * coeff_ii

    # center (r=0) — symmetry: dT/dr = 0 -> d2T/dr2 ≈ 2*(T1 - T0)/dr0^2
    dr0 = r[1] - r[0]
    A[0, 1] = 2.0 * k[0] / (dr0 * dr0)
    A[0, 0] = - A[0, 1]

    # outer node (Robin BC): -k * (Tn - Tn-1)/drl = h_out * (Tn - T_out)
    # rearranged => k/drl * T_{n-1} + (-(k/drl + h)) * T_n + h*T_out = 0
    drl = r[-1] - r[-2]
    A[-1, -2] = k[-1] / drl             # positive
    A[-1, -1] = - (k[-1] / drl + h_out) # negative
    b[-1]    = h_out * T_out

    return A, b

In [82]:
def composite_properties(T, r):
    rho_all, cp_all, k_all = np.zeros_like(T), np.zeros_like(T), np.zeros_like(T)

    for i, (Ti, ri) in enumerate(zip(T, r)):
        if ri < r_interface:  # water region
            rho_i, cp_i, k_i = water_props(Ti)
        elif ri == r_interface:
            rho_w, cp_w, k_w = water_props(Ti)
            rho_al, cp_al, k_al = aluminum_properties(Ti)
            rho_i = 0.5 * (rho_w + rho_al)
            cp_i = 0.5 * (cp_w + cp_al)
            k_i = 0.5 * (k_w + k_al)
        else:  # aluminum wall
            rho_i, cp_i, k_i = aluminum_properties(Ti)
        rho_all[i] = rho_i
        cp_all[i] = cp_i
        k_all[i] = k_i

    return rho_all, cp_all, k_all

def dTdt(t, T):
    print(T)
    rho, cp, k = composite_properties(T, total_nodes)
    A, b = build_A_matrix(total_nodes, k)
    return (-A@T + b) / (rho * cp)

In [84]:
rho, cp, k = composite_properties(orig_T, total_nodes)
A, b = build_A_matrix_cd(total_nodes, k)

print("A[0,:]        =", A[0,:5])
print("A[-1,:]       =", A[-1,-5:])
print("interior row-sums (should be ~0):", np.max(np.abs(A[1:-1].sum(axis=1))))
print("min rho, cp, k:", rho.min(), cp.min(), k.min())

dT0 = dTdt(0.0, orig_T)
print("initial dT/dt min, max:", dT0.min(), dT0.max())

sol = solve_ivp(dTdt, (0, 50), orig_T, method='BDF', max_step=0.1)
print(sol.y)

/tmp/ipython-input-783280215.py:46: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(rho_vals), float(cp_vals), float(k_vals)


A[0,:]        = [-274506.88873022  274506.88873022       0.               0.
       0.        ]
A[-1,:]       = [       0.                0.                0.          3606918.52978015
 -3607874.78592015]
interior row-sums (should be ~0): 7.62939453125e-06
min rho, cp, k: 994.1723894065983 886.3870300000001 0.6218701466020055
[308.15, 308.15, 308.15, 308.15, 308.15, 308.15, 308.15, 308.15, 308.15, 308.15, 308.15, 308.15, 308.15, 308.15, 308.15, 308.15, 308.15, 308.15]
initial dT/dt min, max: -0.014050862095811394 4.099781710137117e-10
[308.15 308.15 308.15 308.15 308.15 308.15 308.15 308.15 308.15 308.15
 308.15 308.15 308.15 308.15 308.15 308.15 308.15 308.15]
[308.15       308.15       308.15       308.15       308.15
 308.15       308.15       308.15       308.15       308.15
 308.15       308.15       308.15       308.15       308.15
 308.15000002 308.14999998 307.4474569 ]
[308.15 308.15 308.15 308.15 308.15 308.15 308.15 308.15 308.15 308.15
 308.15 308.15 308.15 308.15 308.15 30

NotImplementedError: Incoming out of bound